# Imports

In [1]:
#import scanpy as sc
#%matplotlib inline
#import matplotlib.pyplot as mpl
#import matplotlib.colors
import pandas as pd
#import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD
#from plotnine import *
import scipy.stats as ss
import scipy.io as si
import sys, os, argparse
#from sklearn.model_selection import StratifiedShuffleSplit
#from itertools import combinations
#import harmonypy as hm

In [2]:
import gseapy as gp

In [5]:
print(gp.__version__)

1.0.4


In [6]:
def read_geneset(gmt_file):
    genesets = {}
    with open(gmt_file, 'r') as f:
        unsplit = f.read().split('\n')
        for sig in unsplit:
            gene_arr = sig.split('\t')
            if gene_arr[0] != '':
                # first entry is name, skip description, rest are genes
                genesets[gene_arr[0]] = gene_arr[2:]
    return genesets

In [ ]:
cd '/path/to/dir'

In [8]:
hallmarks= read_geneset("h.all.v2023.1.Hs.symbols.gmt")
hms = "h.all.v2023.1.Hs.symbols.gmt"
with open(hms) as gmt:
    print(gmt.read())

HALLMARK_TNFA_SIGNALING_VIA_NFKB	http://www.gsea-msigdb.org/gsea/msigdb/human/geneset/HALLMARK_TNFA_SIGNALING_VIA_NFKB	JUNB	CXCL2	ATF3	NFKBIA	TNFAIP3	PTGS2	CXCL1	IER3	CD83	CCL20	CXCL3	MAFF	NFKB2	TNFAIP2	HBEGF	KLF6	BIRC3	PLAUR	ZFP36	ICAM1	JUN	EGR3	IL1B	BCL2A1	PPP1R15A	ZC3H12A	SOD2	NR4A2	IL1A	RELB	TRAF1	BTG2	DUSP1	MAP3K8	ETS2	F3	SDC4	EGR1	IL6	TNF	KDM6B	NFKB1	LIF	PTX3	FOSL1	NR4A1	JAG1	CCL4	GCH1	CCL2	RCAN1	DUSP2	EHD1	IER2	REL	CFLAR	RIPK2	NFKBIE	NR4A3	PHLDA1	IER5	TNFSF9	GEM	GADD45A	CXCL10	PLK2	BHLHE40	EGR2	SOCS3	SLC2A6	PTGER4	DUSP5	SERPINB2	NFIL3	SERPINE1	TRIB1	TIPARP	RELA	BIRC2	CXCL6	LITAF	TNFAIP6	CD44	INHBA	PLAU	MYC	TNFRSF9	SGK1	TNIP1	NAMPT	FOSL2	PNRC1	ID2	CD69	IL7R	EFNA1	PHLDA2	PFKFB3	CCL5	YRDC	IFNGR2	SQSTM1	BTG3	GADD45B	KYNU	G0S2	BTG1	MCL1	VEGFA	MAP2K3	CDKN1A	CCN1	TANK	IFIT2	IL18	TUBB2A	IRF1	FOS	OLR1	RHOB	AREG	NINJ1	ZBTB10	PLPP3	KLF4	CXCL11	SAT1	CSF1	GPR183	PMEPA1	PTPRE	TLR2	ACKR3	KLF10	MARCKS	LAMB3	CEBPB	TRIP10	F2RL1	KLF9	LDLR	TGIF1	RNF19B	DRAM1	B4GALT1	DNAJB4	CSF2	PDE4B	SNN	PLEK	STAT5

# Reanalysis with updated file

In [18]:
de = pd.read_table('rank_data_GSEA.csv', sep=',')

In [19]:
de = de.drop('Unnamed: 0', axis=1)

In [20]:
eps = 1E-100
de['signed_logfdr'] = np.sign(de['Fold Change (log2)']) * -1 * np.log10(de['FDR P_value'] + eps)

In [21]:
de.to_csv('de.txt', sep='\t', index=False)

In [22]:
de = pd.read_table('de.txt', index_col=0)

In [24]:
de = de.drop(columns=['P-value','Fold Change (log2)','log_pvalue','FDR P_value','rank'])

In [25]:
de

signed_logfdr
Gene                   
CD38         100.000000
HAVCR2       100.000000
EPSTI1        79.570845
TK1           46.202565
KIAA0101      52.451539
...                 ...
SORL1        -31.724409
TCF7         -40.677649
CCR7         -35.389341
S1PR1        -39.220373
IL7R        -100.000000

[12305 rows x 1 columns]

In [26]:
de = de[['signed_logfdr']].reset_index()

In [27]:
de.rename(columns={"Gene": "index"})

index  signed_logfdr
0          CD38     100.000000
1        HAVCR2     100.000000
2        EPSTI1      79.570845
3           TK1      46.202565
4      KIAA0101      52.451539
...         ...            ...
12300     SORL1     -31.724409
12301      TCF7     -40.677649
12302      CCR7     -35.389341
12303     S1PR1     -39.220373
12304      IL7R    -100.000000

[12305 rows x 2 columns]

In [29]:
pre_res = gp.prerank(rnk=de, gene_sets=hms,
                     processes=4,
                     permutation_num=5000, # reduce number to speed up testing
                     outdir='/path/to/outdir', format='png', seed=6)

/var/folders/jz/6g1djx7n1xs4kt2bfqy3wvj00000gp/T/ipykernel_41128/4276622108.py:1: DeprecationWarning: processes is deprecated; use threads
2023-12-30 13:45:38,027 [WARNING] Duplicated values found in preranked stats: 21.58% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


In [30]:
oxphos = pd.DataFrame()

In [31]:
res_oxphos = pre_res.results['HALLMARK_OXIDATIVE_PHOSPHORYLATION']

In [32]:
res_oxphos.keys()

dict_keys(['name', 'es', 'nes', 'pval', 'fdr', 'fwerp', 'tag %', 'gene %', 'lead_genes', 'matched_genes', 'hits', 'RES'])

In [33]:
res_oxphos = pre_res.results['HALLMARK_OXIDATIVE_PHOSPHORYLATION']
oxphos['genes'] = pd.Series(res_oxphos['matched_genes'].split(';'))
oxphos['rank_in_gene_list'] = pd.Series(res_oxphos['hits'])
oxphos['running_es'] = [res_oxphos['RES'][i] for i in res_oxphos['hits']]

In [34]:
oxphos.to_csv('/path/to/outdir')